In [1]:
from dotenv import load_dotenv

load_dotenv()

True

# Defining the tools

In [3]:
import random

def get_weather(location: str) -> float:
    # conexion con una api de clima
    return random.randint(20, 30)

fn_by_name = {f.__name__: f for f in [get_weather]}

In [22]:
fn_by_name

{'get_weather': <function __main__.get_weather(location: str) -> float>}

# Describing the tools

In [13]:
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What is the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)


In [16]:
completion.choices[0].message.__dict__

{'content': None,
 'refusal': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_jbniunBIPTF0CGbM9HC5Hj7w', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]}

In [17]:
messages.append(completion.choices[0].message)

In [30]:
messages

[{'role': 'user', 'content': 'What is the weather like in Paris today?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jbniunBIPTF0CGbM9HC5Hj7w', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')])]

In [31]:
import json

for tool_call in completion.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str(fn_by_name[function_name](**args))
    })



In [32]:
messages

[{'role': 'user', 'content': 'What is the weather like in Paris today?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jbniunBIPTF0CGbM9HC5Hj7w', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_jbniunBIPTF0CGbM9HC5Hj7w',
  'content': '23'}]

In [33]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)


In [35]:
completion.choices[0].message.content

'The current temperature in Paris, France is 23°C.'